# Week 7 Lecture Notes (2/19/25) - Pipes & Signals
demo for hw reference: KEY 
https://github.com/NMcVayZag/SysProg-Class/tree/main/demos/process-thread/5-pipe-handling
- grep is a linux command to search for something

### Pipelines
- pipe: is a pair of file descriptor where ones write and ones read
    - can be used to communicate between processes
    - creating a pipe:
        - ex: int pipefd[2];
              pipe(pipefd)
              - in pipefd[2] are two file descriptors: pipefd[0] is what you can read from and pipefd[1] is what you can write to. (kind of BACKWARDS)
              - pipe format/direction: write-end of pipe(pipefd[1])------> read-end of pipe(pipefd[1])
              - where process_1 will write some stdout to the begining of the pipe that can be read by process_2 at the other end of the pipe - from here on process 2 can use that stdout from process one to complete its mission.
    - copies a file descriptor from one thing to another thing => we can take our pipe and we can copy it to another file descriptor(syscall) or process through the following method: dup2() see screenshot 
        - ex: process1:[stdin, pipefd[1], pipefd[1]] -> "dup2(pipefd[1], stdout)" was called
- creating a pipeline with a supervisor pattern is great as the supervisor simplifies the initial process -> manages overall pipeline or jobs, can chain using pipes or allow to run concurrently. workers between each pipes will be managed by supervisor process.
    - main parent process has to wait till supervisor process is completed => within the supervisor the program will fork again into child processes.
- inside child process one the child will use dup2(pipefd[1], STDOUT_FILENO) to copy writing end of pipe to the processes's standard out => taking the stdout of the child process and connecting it to the write end of the pipe to send it across
    - likewise - this process_2 will dup2(pipefd[0], stdin) to replace stdin with the read end of pipe.
- ONE KEY POINT both processes on ends of pipe inherit both sides of the pipe but can only USE the one they are connected to meaning process_1 inherits pipefd[0](read-end) but cannot interact with or use it and vice-versa with process_2.
- process has a limit of 65,000 file descriptors
- best practice to close a file descriptor when you no longer need it => for instance a supervisor process will inherit pipeline ends but doesn't need them only the child processes that the supervisor oversees needs them.
    - once a end of pipe is copied(duped) into stdin or stdout (becomes the new stdin or stdout) the file descriptior for that end of the pipe can be closed.
- **PIPE(S) are created and called by the supervisor process and connected across child processes.**
- No. of pipes neccessary = N_num_commands - 1;
- if connecting multiple commands you need to loop to create correct number of pipes then loop to create correct number of processes to carry out chain of commands.
    - need to determine in the code which processes are at the beginning of pipeline and at end as they will be using stdin and stdout versus pipe-ends
    - if an error(stderror) is incurred in the middle of the pipeline it will be outputed on the end of the pipeline
- some processes will send out error via stderror and some will send via stdout




